In [1]:
import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import gc, os
import time
from datetime import datetime
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

generate='./data/generate/'
path='./data/rec_data/'

In [2]:
train=pd.read_csv(generate+'train_new01.csv')
val=pd.read_csv(generate+'valid_new01.csv')
test=pd.read_csv(generate+'test_new01.csv')
category=pd.read_csv(generate+'category.csv')
category.rename(columns={"id":"itemId"},inplace=True)
# category10=pd.read_csv(generate+'category10.csv')
# category10.rename(columns={"id":"itemId"},inplace=True)
# category=pd.merge(category,category10,how='left',on='itemId')

data=pd.concat([train,val])
data=pd.concat([data,test])
data=pd.merge(data,category,how='left',on='itemId')

In [3]:
# 造点统计特征

data['pvid_count'] = data.groupby('pvId')['itemId'].transform('count')
data['pvid_item_nunique'] = data.groupby('pvId')['itemId'].transform('nunique')
data['pvid_suv_nunique'] = data.groupby('pvId')['suv'].transform('nunique')

data['item_count'] = data.groupby('itemId')['itemId'].transform('count')
data['item_suv_nunique'] = data.groupby('pvId')['suv'].transform('nunique')

In [4]:
# # 数据处理sparse、dense
# for feat in ['pvId','suv']:
#     lbe = LabelEncoder()
#     data[feat] = lbe.fit_transform(data[feat])

train=data[data['label'].notna()]
val=train[train['day']==3]
train=train[train['day']<3]

test=data[data['label'].isna()]
del data

In [5]:
train.to_csv(generate+'train_new021.csv',index=False)
val.to_csv(generate+'valid_new021.csv',index=False)
data=pd.concat([train,val])
data.to_csv(generate+'train_all021.csv',index=False)
test.to_csv(generate+'test_new021.csv',index=False)